In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.cross_validation import train_test_split
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import confusion_matrix, mean_squared_error, make_scorer

In [2]:
#create train and test set
def split_mask(dftouse, split_size = 0.7):
    itrain, itest = train_test_split(xrange(dftouse.shape[0]), train_size=split_size)
    mask=np.ones(dftouse.shape[0], dtype='int')
    mask[itrain]=1
    mask[itest]=0
    mask = (mask==1)
    return mask

In [3]:
def cv_optimize(clf, parameters, X, y, n_jobs, n_folds, score_func, verbose):
    gs = GridSearchCV(clf, param_grid=parameters, cv=n_folds, n_jobs=n_jobs, verbose=verbose, scoring=score_func)
    gs.fit(X, y)
    print "BEST", gs.best_params_, gs.best_score_, gs.grid_scores_
    best = gs.best_estimator_
    return best, gs.best_params_

def do_classify(clf, parameters, X, y, mask=None, score_func='f1', n_folds=5, n_jobs=1, verbose=False):
    if mask is not None:
        mask = split_mask(X)
    Xtrain, Xtest, ytrain, ytest = X[mask], X[~mask], y[mask], y[~mask]
    clf, best_params_ = cv_optimize(clf, parameters, Xtrain, ytrain, n_jobs=n_jobs, n_folds=n_folds, verbose=verbose, score_func=score_func)
    clf=clf.fit(Xtrain, ytrain)
    train_preds = clf.predict(Xtrain)
    test_preds = clf.predict(Xtest)
    training_accuracy = mean_squared_error(ytrain, train_preds)
    test_accuracy = mean_squared_error(ytest, test_preds)
    print "############# based on standard predict ################"
    print "Accuracy on training data: %0.4f" % (training_accuracy ** 2)
    print "Accuracy on test data:     %0.4f" % (test_accuracy ** 2)
    #print confusion_matrix(ytest, clf.predict(Xtest))
    print "########################################################"
    return clf, Xtrain, ytrain, Xtest, ytest, best_params_

In [5]:
%%time
"""
Import small dataset
"""
df_train_small = pd.read_csv("edited_data/512_features/traindata.csv", index_col=0)
X_small = df_train_small.drop(['smiles'], axis=1)

Wall time: 5min 25s


In [6]:
%%time
"""
Import y-values
"""
df_train_gaps = pd.read_csv("original_data/data_gaps.csv", index_col=0)
y_small = df_train_gaps.loc[X_small.index,:]

Wall time: 2.85 s


In [7]:
print X_small.shape
print y_small.shape

(1000000, 512)
(1000000, 1)


In [8]:
clfForest = RandomForestRegressor()
parameters = {
#    "n_estimators" : [100],
#    "n_estimators" : [100,200,400,800,1600],
    #"n_estimators" : range(1, 21),
#    "max_depth": [25]
#    "max_depth": [25,50,100,200,400]
}

In [9]:
%%time
clfForest, _, _, _, _, best_params_ = do_classify(clfForest, parameters, X_small, np.array(y_small.gap), 
                                                   mask=split_mask(X_small), n_folds = 3, n_jobs = 1, 
                                                   verbose=True, score_func='mean_squared_error')

Fitting 3 folds for each of 1 candidates, totalling 3 fits
BEST {} -0.00751386494152 [mean: -0.00751, std: 0.00001, params: {}]
############# based on standard predict ################
Accuracy on training data: 0.0000
Accuracy on test data:     0.0000
########################################################
Wall time: 1h 11min 12s


[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed: 34.5min finished


In [10]:
def write_to_file(filename, predictions):
    with open(filename, "w") as f:
        f.write("Id,Prediction\n")
        for i,p in enumerate(predictions):
            f.write(str(i+1) + "," + str(p) + "\n")

In [11]:
%xdel df_train_small
%xdel df_train_gaps
%xdel X_small
%xdel y_small

In [13]:
test = pd.read_csv("edited_data/512_features/testdata.csv", index_col=0)

In [15]:
test.drop(['smiles'], axis=1, inplace=True)
test.head()

,0,1,2,3,4,5,6,7,8,9,...,502,503,504,505,506,507,508,509,510,511
0,1,1,0,0,1,0,1,1,1,1,...,1,1,1,1,0,1,0,0,1,1
1,1,1,0,0,1,1,1,1,1,1,...,1,0,1,1,0,1,0,0,1,1
2,1,1,1,1,1,1,1,1,0,1,...,1,0,1,1,1,1,1,0,1,1
3,0,1,1,1,0,1,1,0,0,1,...,1,0,1,1,0,1,1,0,0,1
4,1,0,0,0,1,1,1,1,1,1,...,1,0,1,1,1,1,1,0,1,1


In [16]:
preds = clfForest.predict(test)
write_to_file("submissions/RF_2.csv", preds)